In [0]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [52]:
raw_df = pd.read_excel('openfoodfacts_simplified_database.xlsx')
raw_df = raw_df.set_index('product_name')
raw_df.head()

,brands,categories,stores,countries,allergens,additives_n,additives_tags,nutrition_grade_fr,nova_group,pnns_groups_1,pnns_groups_2,energy_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
product_name,,,,,,,,,,,,,,,,,,,
Les Ã©vasions Poulet Satay et son riz,"Les Ã‰vasions,Carrefour","Plats prÃ©parÃ©s,Produits Ã la viande,Plats p...","Carrefour Market,Carrefour",France,"en:gluten,en:peanuts,en:soybeans",1.0,en:e150a,b,4.0,Composite foods,One-dish meals,590.0,0.5,5.5,1.8,6.2,0.236,0.0,0.0
Haricots verts extra-fins,Carrefour,NaN,Carrefour,en:france,NaN,0.0,NaN,NaN,NaN,unknown,unknown,94.0,NaN,1.0,2.8,1.6,0.136,NaN,NaN
Pommes Ã dipper,Carrefour,Pommes de terre prÃ©frites surgelÃ©es,Carrefour,"France,Espagne",NaN,4.0,"en:e14xx,en:e415,en:e450,en:e450i,en:e500",NaN,4.0,Cereals and potatoes,Potatoes,611.0,0.6,NaN,4.8,2.3,0.240,NaN,NaN
VeloutÃ© de LÃ©gumes,carrefour,Soupes de lÃ©gumes dÃ©shydratÃ©es,Carrefour,France,"en:celery,en:gluten,en:milk",NaN,NaN,b,NaN,Fruits and vegetables,Soups,1477.0,1.5,10.0,9.9,5.4,2.200,1.0,1.0
jus de pomme,Carrefour,"Boissons, Aliments et boissons Ã base de vÃ©g...",Carrefour,France,NaN,NaN,NaN,c,NaN,Beverages,Fruit juices,176.0,0.0,9.7,0.0,0.5,0.000,3.0,-3.0


In [53]:
raw_df = raw_df.replace(['a','b','c','d','e'], [0,1,2,3,4])
raw_df.head()

,brands,categories,stores,countries,allergens,additives_n,additives_tags,nutrition_grade_fr,nova_group,pnns_groups_1,pnns_groups_2,energy_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
product_name,,,,,,,,,,,,,,,,,,,
Les Ã©vasions Poulet Satay et son riz,"Les Ã‰vasions,Carrefour","Plats prÃ©parÃ©s,Produits Ã la viande,Plats p...","Carrefour Market,Carrefour",France,"en:gluten,en:peanuts,en:soybeans",1.0,en:e150a,1.0,4.0,Composite foods,One-dish meals,590.0,0.5,5.5,1.8,6.2,0.236,0.0,0.0
Haricots verts extra-fins,Carrefour,NaN,Carrefour,en:france,NaN,0.0,NaN,NaN,NaN,unknown,unknown,94.0,NaN,1.0,2.8,1.6,0.136,NaN,NaN
Pommes Ã dipper,Carrefour,Pommes de terre prÃ©frites surgelÃ©es,Carrefour,"France,Espagne",NaN,4.0,"en:e14xx,en:e415,en:e450,en:e450i,en:e500",NaN,4.0,Cereals and potatoes,Potatoes,611.0,0.6,NaN,4.8,2.3,0.240,NaN,NaN
VeloutÃ© de LÃ©gumes,carrefour,Soupes de lÃ©gumes dÃ©shydratÃ©es,Carrefour,France,"en:celery,en:gluten,en:milk",NaN,NaN,1.0,NaN,Fruits and vegetables,Soups,1477.0,1.5,10.0,9.9,5.4,2.200,1.0,1.0
jus de pomme,Carrefour,"Boissons, Aliments et boissons Ã base de vÃ©g...",Carrefour,France,NaN,NaN,NaN,2.0,NaN,Beverages,Fruit juices,176.0,0.0,9.7,0.0,0.5,0.000,3.0,-3.0


In [0]:
features = ['energy_100g','saturated-fat_100g','sugars_100g','fiber_100g','proteins_100g','sodium_100g']
label = ['nutrition_grade_fr']

In [55]:
raw_df = raw_df[features + label]
raw_df.head()

,energy_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,nutrition_grade_fr
product_name,,,,,,,
Les Ã©vasions Poulet Satay et son riz,590.0,0.5,5.5,1.8,6.2,0.236,1.0
Haricots verts extra-fins,94.0,NaN,1.0,2.8,1.6,0.136,NaN
Pommes Ã dipper,611.0,0.6,NaN,4.8,2.3,0.240,NaN
VeloutÃ© de LÃ©gumes,1477.0,1.5,10.0,9.9,5.4,2.200,1.0
jus de pomme,176.0,0.0,9.7,0.0,0.5,0.000,2.0


In [56]:
raw_df = raw_df.dropna()
raw_df.head()

,energy_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,nutrition_grade_fr
product_name,,,,,,,
Les Ã©vasions Poulet Satay et son riz,590.0,0.5,5.5,1.8,6.2,0.236,1.0
VeloutÃ© de LÃ©gumes,1477.0,1.5,10.0,9.9,5.4,2.200,1.0
jus de pomme,176.0,0.0,9.7,0.0,0.5,0.000,2.0
Fingers de Poulet,1022.0,1.9,0.5,1.4,12.0,0.332,1.0
Orange Jus Ã base de concentrÃ©,196.0,0.0,11.0,0.5,0.7,0.040,2.0


In [57]:
X = raw_df[features]
X.head()

,energy_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g
product_name,,,,,,
Les Ã©vasions Poulet Satay et son riz,590.0,0.5,5.5,1.8,6.2,0.236
VeloutÃ© de LÃ©gumes,1477.0,1.5,10.0,9.9,5.4,2.200
jus de pomme,176.0,0.0,9.7,0.0,0.5,0.000
Fingers de Poulet,1022.0,1.9,0.5,1.4,12.0,0.332
Orange Jus Ã base de concentrÃ©,196.0,0.0,11.0,0.5,0.7,0.040


In [58]:
y = raw_df[label]
y.head()

,nutrition_grade_fr
product_name,
Les Ã©vasions Poulet Satay et son riz,1.0
VeloutÃ© de LÃ©gumes,1.0
jus de pomme,2.0
Fingers de Poulet,1.0
Orange Jus Ã base de concentrÃ©,2.0


In [0]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [0]:
def runXGB(train_X, train_y, validation_X, validation_y, test_X):
    param = {}
    param['objective'] = 'multi:softmax'
    param['num_class'] = 5
    param['eta'] = 0.01
    param['max_depth'] = 6
    param['silent'] = 1
    param['gamma'] = 1
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['max_delta_step'] = 0
    param['subsample'] = 1
    param['colsample_bytree'] = 1
    param['lambda'] = 1
    param['alpha'] = 0
    param['seed'] = 0
    num_rounds = 1000

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label = train_y)
    xgcv = xgb.DMatrix(validation_X, label = validation_y)
    xgtest = xgb.DMatrix(test_X)

    evallist = [(xgcv,'eval')]
    model = xgb.train(plst, xgtrain, num_rounds, evallist, early_stopping_rounds = 10)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [61]:
preds, model = runXGB(X_train, y_train, X_val, y_val, X_test)

[0]	eval-mlogloss:1.59637
Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	eval-mlogloss:1.58341
[2]	eval-mlogloss:1.57068
[3]	eval-mlogloss:1.55825
[4]	eval-mlogloss:1.54602
[5]	eval-mlogloss:1.53405
[6]	eval-mlogloss:1.52221
[7]	eval-mlogloss:1.51062
[8]	eval-mlogloss:1.49922
[9]	eval-mlogloss:1.48805
[10]	eval-mlogloss:1.47708
[11]	eval-mlogloss:1.46622
[12]	eval-mlogloss:1.45562
[13]	eval-mlogloss:1.4452
[14]	eval-mlogloss:1.43495
[15]	eval-mlogloss:1.42479
[16]	eval-mlogloss:1.41493
[17]	eval-mlogloss:1.40516
[18]	eval-mlogloss:1.39554
[19]	eval-mlogloss:1.3861
[20]	eval-mlogloss:1.37685
[21]	eval-mlogloss:1.36774
[22]	eval-mlogloss:1.35883
[23]	eval-mlogloss:1.35006
[24]	eval-mlogloss:1.34153
[25]	eval-mlogloss:1.33311
[26]	eval-mlogloss:1.32482
[27]	eval-mlogloss:1.31648
[28]	eval-mlogloss:1.30828
[29]	eval-mlogloss:1.30038
[30]	eval-mlogloss:1.29238
[31]	eval-mlogloss:1.28449
[32]	eval-mlogloss:1.27695
[33]	eval-mlogloss:1.26929
[34]	eval-mlogloss:1.26195
[35]	e

In [71]:
(y_test['nutrition_grade_fr'] == preds).mean()

0.8322903629536921